In [ ]:
import os
import numpy as np

# Obtener el directorio actual del script
directorio_actual = os.path.abspath(os.path.dirname(__file__))


def swell_generator():
    for i in range(3601):
        nombre_archivo = f"or_s_{i}.npy"
        ruta_archivo = os.path.join(directorio_actual, 'original_signals', nombre_archivo)

        if os.path.exists(ruta_archivo):
            print(f"Se encontró el archivo {nombre_archivo}")
            # Cargar los datos desde el archivo .npy
            data = np.load(ruta_archivo)
            # Normalizar la señal al rango [-1, 1]
            signal_original = data / np.sqrt(np.mean(data**2))

            # Parámetros de la amplificación aleatoria
            longitud_signal = len(signal_original)
            duracion_amplificacion = 3000  # en muestras (0.3 segundos a una tasa de muestreo de 10,000 Hz)
            factor_amplitud_min = 1.2
            factor_amplitud_max = 1.7

            # Generar la posición aleatoria para la amplificación
            inicio_amplificacion = np.random.randint(0, longitud_signal - duracion_amplificacion)

            # Generar el factor de amplificación aleatorio
            factor_amplitud = np.random.uniform(factor_amplitud_min, factor_amplitud_max)

            # Aplicar la amplificación en la parte aleatoria de la señal original
            signal_amplificada = np.copy(signal_original)
            signal_amplificada[inicio_amplificacion:(inicio_amplificacion + duracion_amplificacion)] *= factor_amplitud

            # Implementar la forma de onda del Swell (simétrica)
            angulo_rampa = np.pi / 2  # Pendiente de la rampa (45 grados)
            rampa_arriba = np.linspace(0, 1, duracion_amplificacion // 2)
            rampa_abajo = np.linspace(1, 0, duracion_amplificacion // 2)
            senal_swell = np.concatenate((rampa_arriba, rampa_abajo))
            senal_swell = np.repeat(senal_swell, repeats=int(np.ceil(len(signal_amplificada) / len(senal_swell))))

            # Aplicar la forma de onda del Swell a la señal amplificada
            signal_con_swell = signal_amplificada * senal_swell

            # Guardar la señal con Swell en un nuevo archivo
            directorio_destino = os.path.join(directorio_actual, 'swell_signals')
            os.makedirs(directorio_destino, exist_ok=True)
            nombre_archivo_nuevo = f"sw_s_{i}.npy"
            ruta_archivo_nuevo = os.path.join(directorio_destino, nombre_archivo_nuevo)
            np.save(ruta_archivo_nuevo, signal_con_swell)
            print(f"Señal con Swell guardada en {nombre_archivo_nuevo}")
        else:
            print(f"No se encontró el archivo {nombre_archivo}")


swell_generator()
